# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:

weather_df = pd.read_csv("../starter_code/Weatherpy.csv", encoding="utf-8")
weather_df.head()


,City,City.1,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,0,victoria,HK,22.2855,114.1577,1623262696,26,89,301.66,2.68
1,1,sitka,US,57.0531,-135.3300,1623262860,75,82,284.09,4.63
2,2,kumluca,TR,36.3703,30.2869,1623262861,2,80,295.04,0.14
3,3,mataura,NZ,-46.1927,168.8643,1623262861,85,91,282.86,1.01
4,4,padang,ID,-0.9492,100.3543,1623262862,72,87,298.01,1.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [24]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

#Add Heatmap layer to map.
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Narrow down cities to fit weather conditions
ideal_weather_df = weather_df.loc[(weather_df["Max Temperature"] > 70) & (weather_df["Max Temperature"] < 80) & (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] < 7) & (weather_df["Humidity"] < 80), :]

# Drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)



# Display df
ideal_weather_df

,index,City,City.1,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotels_df = ideal_weather_df.loc

for i in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[i]['Latitude']
    lng = ideal_weather_df.loc[i]['Longitude']

    paramaters = {"location": f"{lat},{lng}", "radius": 5000, "types" : "hotel", "key": gkey}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=paramaters)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
        
# Add a "Hotel Name" column to the DataFrame
ideal_weather_df["Hotel Name"] = hotels
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df

,index,City,City.1,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))